Steps¶
1. Identify and gather relevant data
2. Build and evaluate the spam email classifier
3. Build an improved classifier?

In [2]:
# Find a spam dataset at https://huggingface.co/datasets and load it using the datasets library

from datasets import load_dataset

dataset = load_dataset("sms_spam", split=["train"])[0]

for entry in dataset.select(range(3)):
    sms = entry["sms"]
    label = entry["label"]
    print(f"label={label}, sms={sms}")

/Users/yanyan/personal-github/workspace-udacity/generative-ai-nanodegree/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


label=0, sms=Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...

label=0, sms=Ok lar... Joking wif u oni...

label=1, sms=Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's



In [3]:
# Convenient dictionaries to convert between labels and ids
id2label = {0: "NOT SPAM", 1: "SPAM"}
label2id = {"NOT SPAM": 0, "SPAM": 1}

for entry in dataset.select(range(3)):
    sms = entry["sms"]
    label_id = entry["label"]
    print(f"label={id2label[label_id]}, sms={sms}")

label=NOT SPAM, sms=Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...

label=NOT SPAM, sms=Ok lar... Joking wif u oni...

label=SPAM, sms=Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's



In [8]:
# Let's start with this helper function that will help us format sms messages
# for the LLM.
def get_sms_messages_string(dataset, item_numbers_range, include_labels=False):
    sms_messages_string = ""
    for item_number, entry in zip(item_numbers_range, dataset.select(item_numbers_range)):
        sms = entry["sms"]
        label_id = entry["label"]

        if include_labels:
            sms_messages_string += (
                f"{item_number} (label={id2label[label_id]}) -> {sms}\n"
            )
        else:
            sms_messages_string += f"{item_number} -> {sms}\n"

    return sms_messages_string


print(get_sms_messages_string(dataset, range(3), include_labels=True))

0 (label=NOT SPAM) -> Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...

1 (label=NOT SPAM) -> Ok lar... Joking wif u oni...

2 (label=SPAM) -> Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's




In [9]:
# Get a few messages and format them as a string
sms_messages_string = get_sms_messages_string(dataset, range(7, 15))

# Construct a query to send to the LLM including the sms messages.
# Ask it to respond in JSON format.
query = f"""
{sms_messages_string}
---
Classify the messages above as SPAM or NOT SPAM. Respond in JSON format.
Use the following format: {{"0": "NOT SPAM", "1": "SPAM"}}
"""

print(query)


7 -> As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune

8 -> WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.

9 -> Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030

10 -> I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.

11 -> SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info

12 -> URGENT! You have won a 1 week FREE membership in our £100,000 Prize Jackpot! Txt the word: CLAIM to No: 81010 T&C www.dbuk.net LCCLTD POBOX 4403LDNW1A7RW18

13 -> I've been searching for the right words to thank you for this breather. I pro

In [12]:
# Replace <MASK> with your LLMs response

response = {
    "7": "NOT SPAM",
    "8": "SPAM",
    "9": "SPAM",
    "10": "NOT SPAM",
    "11": "SPAM",
    "12": "SPAM",
    "13": "NOT SPAM",
    "14": "NOT SPAM"
}


In [14]:
# Estimate the accuracy of your classifier by comparing your responses to the labels in the dataset


def get_accuracy(response, dataset, original_indices):
    correct = 0
    total = 0

    for entry_number, prediction in response.items():
        if int(entry_number) not in original_indices:
            continue

        label_id = dataset[int(entry_number)]["label"]
        label = id2label[label_id]

        # If the prediction from the LLM matches the label in the dataset
        # we increment the number of correct predictions.
        # (Since LLMs do not always produce the same output, we use the
        # lower case version of the strings for comparison)
        if prediction.lower() == label.lower():
            correct += 1

        # increment the total number of predictions
        total += 1

    try:
        accuracy = correct / total
    except ZeroDivisionError:
        print("No matching results found!")
        return

    return round(accuracy, 2)


print(f"Accuracy: {get_accuracy(response, dataset, range(7, 15))}")

Accuracy: 1.0


In [16]:
# Get a few labelled messages and format them as a string
sms_messages_string_w_labels = get_sms_messages_string(
    dataset, range(54, 60), include_labels=True
)

# Get a few unlabelled messages and format them as a string
sms_messages_string_no_labels = get_sms_messages_string(dataset, range(7, 15))

# Construct a query to send to the LLM including the labelled messages
# as well as the unlabelled messages. Ask it to respond in JSON format
query = f"""
{sms_messages_string_w_labels}
{sms_messages_string_no_labels}
---
Classify the messages above as SPAM or NOT SPAM. Respond in JSON format.
Use the following format: {{"0": "NOT SPAM", "1": "SPAM"}}.
Some examples have been labeled for you.
"""

print(query)


54 (label=SPAM) -> SMS. ac Sptv: The New Jersey Devils and the Detroit Red Wings play Ice Hockey. Correct or Incorrect? End? Reply END SPTV

55 (label=NOT SPAM) -> Do you know what Mallika Sherawat did yesterday? Find out now @  &lt;URL&gt;

56 (label=SPAM) -> Congrats! 1 year special cinema pass for 2 is yours. call 09061209465 now! C Suprman V, Matrix3, StarWars3, etc all 4 FREE! bx420-ip4-5we. 150pm. Dont miss out! 

57 (label=NOT SPAM) -> Sorry, I'll call later in meeting.

58 (label=NOT SPAM) -> Tell where you reached

59 (label=NOT SPAM) -> Yes..gauti and sehwag out of odi series.


7 -> As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune

8 -> WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.

9 -> Had your mobile 11 months or more? U R entitled to Update to the la

In [17]:
response = {
    "7": "NOT SPAM",
    "8": "SPAM",
    "9": "SPAM",
    "10": "NOT SPAM",
    "11": "SPAM",
    "12": "SPAM",
    "13": "NOT SPAM",
    "14": "NOT SPAM"
}

In [19]:
print(f"Accuracy: {get_accuracy(response, dataset, range(7, 15)):.2f}")

Accuracy: 1.00


In [20]:
# Show the messages that were misclassified, if you have any

def print_misclassified_messages(response, dataset):
    for entry_number, prediction in response.items():
        label_id = dataset[int(entry_number)]["label"]
        label = id2label[label_id]

        if prediction.lower() != label.lower():
            sms = dataset[int(entry_number)]["sms"]
            print("---")
            print(f"Message: {sms}")
            print(f"Label: {label}")
            print(f"Prediction: {prediction}")


print_misclassified_messages(response, dataset)

In [1]:
20/2.0

10.0

In [2]:
(1 == 2) or (2 == 2)

True

In [4]:
x = 13.37
x.islower()


AttributeError: 'float' object has no attribute 'islower'